<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará artículos deportivos de la bbc para generar los embeddings, es decir, que los vectores tendrán la forma en función de las notas periodísticas realizadas por la bbc en su sección deportiva

#### Importamos las librerias necesarias

In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import multiprocessing
from gensim.models import Word2Vec

# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np        

In [3]:
#Si se utiliza google drive en colab
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


### Carga de dataset para poesterior procesamiento
Utilizaremos como dataset artículos deportivos de la bbc.
Consta de 471 artículos de la bbc.

#### Nos posicionamos en la carpeta

In [6]:
# Me posicino en la carpeta con el conjunto de articulos y listo los archivos
path_base = "./bbc_sports_articles/"
#Utilizar esta linea en caso de utilizar Colab
#path_base = "/content/drive/MyDrive/NLP/bbc_sports_articles/"
file_names = os.listdir(path_base)
# Muestro los 10 primeros nombres de los archivos
file_names[:10]

['213.txt',
 '212.txt',
 '215.txt',
 '220.txt',
 '219.txt',
 '217.txt',
 '216.txt',
 '218.txt',
 '211.txt',
 '201.txt']

#### Generamos una variables única a partir de tods los archivos de texto que representan los artículos

In [8]:
articles=""

for file in file_names:
    file = open(path_base+file, mode="rt", encoding='utf-8')
    text=file.read()
    file.close()
    articles +=text 

In [9]:
articles

'South Africa recall Boje\n\nSouth Africa have recalled spinner Nicky Boje for the first Test against England, which begins on 17 December.\n\nBoje missed the recent tour of India because of fears he would be called in for questioning by Indian police over match-fixing allegations. Pace bowler Dale Steyn and opening batsmen AB de Villiers are new faces in a 13-man squad. Despite suffering poor form in India, new wicket-keeper Thami Tsolekile has been preferred to Mark Boucher. There is no place for opener Herschelle Gibbs, who has been struggling with a finger injury after missing the India trip, and spin all-rounder Justin Ontong has been dropped. Nicky Boje, who underwent minor surgery to remove a growth in his neck, will have to pass a fitness test before the match. Port Elizabeth, where the first Test is being held, is South Africa\'s most spin-friendly venue.\n\nGraeme Smith (Capt), Nicky Boje, Hashim Amla, Zander de Bruyn, AB de Villiers, Boeta Dippenaar, Andrew Hall, Jacques Kal

#### Separamos el texto según los párrafos y quitamos párrafos vacios

In [11]:
lista_articulos = articles.split("\n")
lista_articulos = list(filter(None, lista_articulos))
lista_articulos

['South Africa recall Boje',
 'South Africa have recalled spinner Nicky Boje for the first Test against England, which begins on 17 December.',
 "Boje missed the recent tour of India because of fears he would be called in for questioning by Indian police over match-fixing allegations. Pace bowler Dale Steyn and opening batsmen AB de Villiers are new faces in a 13-man squad. Despite suffering poor form in India, new wicket-keeper Thami Tsolekile has been preferred to Mark Boucher. There is no place for opener Herschelle Gibbs, who has been struggling with a finger injury after missing the India trip, and spin all-rounder Justin Ontong has been dropped. Nicky Boje, who underwent minor surgery to remove a growth in his neck, will have to pass a fitness test before the match. Port Elizabeth, where the first Test is being held, is South Africa's most spin-friendly venue.",
 'Graeme Smith (Capt), Nicky Boje, Hashim Amla, Zander de Bruyn, AB de Villiers, Boeta Dippenaar, Andrew Hall, Jacques 

#### Armamos el dataset a partir de la lista de articulos

In [12]:
df  = pd.DataFrame(lista_articulos)
df.head()

,0
0,South Africa recall Boje
1,South Africa have recalled spinner Nicky Boje ...
2,Boje missed the recent tour of India because o...
3,"Graeme Smith (Capt), Nicky Boje, Hashim Amla, ..."
4,Flintoff fit to bowl at Wanderers


In [13]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2645


### 1 - Preprocesamiento

Hacemos el preprocesamiento del texto

In [14]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [15]:
# Vemos los tokens de algunas oraciones
sentence_tokens[:2]

[['south', 'africa', 'recall', 'boje'],
 ['south',
  'africa',
  'have',
  'recalled',
  'spinner',
  'nicky',
  'boje',
  'for',
  'the',
  'first',
  'test',
  'against',
  'england',
  'which',
  'begins',
  'on',
  '17',
  'december']]

### 2 - Crear los vectores (word2vec)

In [16]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

#### Creamos un modelo con skipgram

In [85]:
# Crearmos el modelo generador de vectores con skigram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

#### Creamos un modelo con CBOW

In [86]:
w2v_model_cbow = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [87]:
# Buildeamos el vocabulario con los tokens para los 2 modelos
w2v_model.build_vocab(sentence_tokens)
w2v_model_cbow.build_vocab(sentence_tokens)

In [88]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2645


In [89]:
# Cantidad de words encontradas en el corpus words
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 3437


### 3 - Entrenar el modelo generador

In [90]:
# Entrenamos el modelo generador de vectores con skigram
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=400,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1617518.125
Loss after epoch 1: 1096791.375
Loss after epoch 2: 1014552.75
Loss after epoch 3: 958408.25
Loss after epoch 4: 908472.0
Loss after epoch 5: 895072.0
Loss after epoch 6: 879106.0
Loss after epoch 7: 862813.5
Loss after epoch 8: 813938.0
Loss after epoch 9: 791992.0
Loss after epoch 10: 780203.0
Loss after epoch 11: 766857.0
Loss after epoch 12: 757141.0
Loss after epoch 13: 746532.0
Loss after epoch 14: 737251.0
Loss after epoch 15: 729934.0
Loss after epoch 16: 722112.0
Loss after epoch 17: 714761.0
Loss after epoch 18: 709239.0
Loss after epoch 19: 671828.0
Loss after epoch 20: 643748.0
Loss after epoch 21: 638758.0
Loss after epoch 22: 635062.0
Loss after epoch 23: 627544.0
Loss after epoch 24: 621380.0
Loss after epoch 25: 622494.0
Loss after epoch 26: 616376.0
Loss after epoch 27: 617358.0
Loss after epoch 28: 613484.0
Loss after epoch 29: 608644.0
Loss after epoch 30: 610324.0
Loss after epoch 31: 606940.0
Loss after epoch 32: 606650.0
Loss after 

(46816311, 65325600)

In [91]:
# Entrenamos el modelo generador de vectores con cbow
w2v_model_cbow.train(sentence_tokens,
                 total_examples=w2v_model_cbow.corpus_count,
                 epochs=400,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 669469.9375
Loss after epoch 1: 468591.3125
Loss after epoch 2: 415281.0
Loss after epoch 3: 393540.125
Loss after epoch 4: 350658.375
Loss after epoch 5: 323135.5
Loss after epoch 6: 312880.25
Loss after epoch 7: 304252.0
Loss after epoch 8: 295718.25
Loss after epoch 9: 287715.75
Loss after epoch 10: 279872.25
Loss after epoch 11: 259556.75
Loss after epoch 12: 246164.0
Loss after epoch 13: 240317.5
Loss after epoch 14: 234108.5
Loss after epoch 15: 229041.5
Loss after epoch 16: 223915.0
Loss after epoch 17: 218983.5
Loss after epoch 18: 213787.0
Loss after epoch 19: 209814.0
Loss after epoch 20: 205473.0
Loss after epoch 21: 202741.0
Loss after epoch 22: 198822.5
Loss after epoch 23: 195379.5
Loss after epoch 24: 191791.0
Loss after epoch 25: 189698.5
Loss after epoch 26: 186371.0
Loss after epoch 27: 185405.5
Loss after epoch 28: 182337.0
Loss after epoch 29: 179345.5
Loss after epoch 30: 178428.5
Loss after epoch 31: 169956.5
Loss after epoch 32: 159106.0
Loss 

(46816311, 65325600)

### 4 - Ensayar

Ensayamos los modelos de cbow y skip-gram para ver si dan resultados similares o diferentes

In [93]:
palabra = "federer"
print("Palabras que mas se relacionan con "+palabra+ " en skip-gram")
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con federer en skip-gram


[('roger', 0.3779347836971283),
 ('safin', 0.3191946744918823),
 ('moya', 0.28676560521125793),
 ('dawson', 0.27054911851882935),
 ('joe', 0.2684449553489685),
 ('farrell', 0.26585882902145386),
 ('hewitt', 0.26484522223472595),
 ('tsolekile', 0.2613486647605896),
 ('ljubicic', 0.2590571343898773),
 ('swiss', 0.258991003036499)]

In [94]:
print("Palabras que mas se relacionan con "+palabra+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con federer en CBOW


[('ljubicic', 0.29153430461883545),
 ('luckhurst', 0.27406787872314453),
 ('moya', 0.2650642991065979),
 ('russian', 0.26134949922561646),
 ('safin', 0.25648677349090576),
 ('dawson', 0.25262653827667236),
 ('joe', 0.24905958771705627),
 ('henman', 0.24786007404327393),
 ('hewitt', 0.24012070894241333),
 ('tsolekile', 0.2387615442276001)]

In [95]:
palabra = "tennis"
print("Palabras que menos se relacionan con "+palabra+ " en skip-gram")
w2v_model.wv.most_similar(negative=[palabra], topn=10)

Palabras que menos se relacionan con tennis en skip-gram


[('lead', 0.08152613043785095),
 ('gough', 0.07745885848999023),
 ('keep', 0.05492449179291725),
 ('breaks', 0.05271901935338974),
 ("o'connell", 0.04981561005115509),
 ('knock', 0.045716069638729095),
 ('nadal', 0.04451170936226845),
 ('33', 0.0434638075530529),
 ('period', 0.042993493378162384),
 ('penalty', 0.041573844850063324)]

In [96]:
print("Palabras que menos se relacionan con "+palabra+ " en CBOW")
w2v_model_cbow.wv.most_similar(negative=[palabra], topn=10)

Palabras que menos se relacionan con tennis en CBOW


[('if', 0.19891873002052307),
 ('choose', 0.18302175402641296),
 ('period', 0.18245349824428558),
 ('send', 0.1791364699602127),
 ('commented', 0.1791347861289978),
 ('27', 0.176137775182724),
 ('thanou', 0.173133984208107),
 ('luciano', 0.17285612225532532),
 ('dimitri', 0.1702617108821869),
 ('simpson', 0.1659807413816452)]

In [97]:
print("Palabras que mas se relacionan con "+palabra+ " en skip-gram")
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con tennis en skip-gram


[('anti', 0.28657257556915283),
 ('intense', 0.2819055914878845),
 ('manager', 0.27050110697746277),
 ('renewed', 0.26637521386146545),
 ('critical', 0.26584944128990173),
 ('mixed', 0.2622706890106201),
 ('duty', 0.2612261176109314),
 ('unbeaten', 0.25997716188430786),
 ('contenders', 0.25989288091659546),
 ('ak', 0.25971633195877075)]

In [98]:
print("Palabras que mas se relacionan con "+palabra+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con tennis en CBOW


[('cricket', 0.24108010530471802),
 ('critical', 0.21610483527183533),
 ('unbeaten', 0.21249520778656006),
 ('rugby', 0.20668523013591766),
 ('shape', 0.20274102687835693),
 ('considering', 0.19759802520275116),
 ('sports', 0.19675494730472565),
 ('entry', 0.19075793027877808),
 ("you're", 0.18574470281600952),
 ('cross', 0.1849265843629837)]

In [99]:
palabra = "rugby"
print("Palabras que mas se relacionan con "+palabra+ " en skip-gram")
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con rugby en skip-gram


[('hinshelwood', 0.39082202315330505),
 ('league', 0.35229426622390747),
 ('football', 0.3438699543476105),
 ("union's", 0.3429357409477234),
 ('dunbar', 0.3334376811981201),
 ('hines', 0.32767388224601746),
 ('union', 0.3156810998916626),
 ('ross', 0.31273823976516724),
 ('lamont', 0.31239473819732666),
 ('absolutely', 0.30008453130722046)]

In [100]:
print("Palabras que mas se relacionan con "+palabra+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con rugby en CBOW


[('cricket', 0.3122709095478058),
 ('norwich', 0.24848157167434692),
 ('territory', 0.24679064750671387),
 ('strength', 0.230016827583313),
 ('stars', 0.22130823135375977),
 ('tennis', 0.20668521523475647),
 ('saints', 0.20396223664283752),
 ('reigning', 0.19717037677764893),
 ('raise', 0.1909572333097458),
 ('legend', 0.18627355992794037)]

In [101]:
palabra = "opponent"
print("Palabras que menos se relacionan con "+palabra+ " en skip-gram")
w2v_model.wv.most_similar(negative=[palabra], topn=10)

Palabras que menos se relacionan con opponent en skip-gram


[('bangladesh', 0.07219025492668152),
 ('bring', 0.05842190608382225),
 ('corner', 0.05435711890459061),
 ('men', 0.053290702402591705),
 ("friday's", 0.04594102501869202),
 ('individual', 0.04236624762415886),
 ("women's", 0.04060395061969757),
 ('battle', 0.03602300211787224),
 ('visit', 0.03146599233150482),
 ('olympics', 0.030490005388855934)]

In [102]:
print("Palabras que menos se relacionan con "+palabra+ " en CBOW")
w2v_model_cbow.wv.most_similar(negative=[palabra], topn=10)

Palabras que menos se relacionan con opponent en CBOW


[('athletes', 0.20870736241340637),
 ('your', 0.1832941472530365),
 ("women's", 0.18156874179840088),
 ('his', 0.18141065537929535),
 ('the', 0.17458245158195496),
 ('favourites', 0.1713891625404358),
 ('continue', 0.16974130272865295),
 ("couldn't", 0.16576015949249268),
 ('than', 0.16427287459373474),
 ('their', 0.16394923627376556)]

Otras palabras interesantes probadas:


*   ceremony
*   champions
*   fitness
*   greatest
*   leader


### 5 - Visualizar agrupación de vectores

#### Definimos nuestra funcion para reducir dimensiones con T-SNE

In [106]:
def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, perplexity=15,random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

#### Graficamos para el modelo con skip-gram

In [107]:
x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=350
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS],
                 title="Representacion en 2 dim de T-SNE para modelo Skip-Gram")
fig.show(renderer="colab")

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



<img src="images/TSNE-skipgram.png" />

#### Graficamos para el modelo con CBOW

In [108]:
x_vals, y_vals, labels = reduce_dimensions(w2v_model_cbow)

MAX_WORDS=350
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS],
                 title="Representacion en 2 dim de T-SNE para modelo CBOW")
fig.show(renderer="colab")

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



<img src="images/TSNE-cbow.png" />

### 6 - Conclusiones

Con este ejercicio pudimos ver la generacion de un embeding para un corpus.

Se probaron tanto el método de CBOW como el de skip-gram a través de la librería GENSIM.

En función del dataset de artículos utilizado y las pruebas realizadas, obtuvimos mejores resultados con el método de Skip-Gram. El mismo parece haber encontrado mejores relaciones en las palabras con el contexto dado. Igualmente ambos algoritmos dieron buenos resultados.